In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('../input/44352/training_solutions_rev1.csv')

train_ratio = 0.8
validation_ratio = 0
test_ratio = 0.20

df_train, df_test = train_test_split(df, test_size=1 - train_ratio)

# # test is now 15% of the initial data set
# # validation is now 10% of the initial data set
# df_val, df_test = train_test_split(df_test, test_size=test_ratio/(test_ratio + validation_ratio)) 

df_train.shape, df_test.shape

In [ ]:
import skimage.transform 
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
          

ORIG_SHAPE = (424,424)
CROP_SHAPE = (256,256)
IMG_SHAPE = (64,64)

def rotate(img, angle):
    return skimage.transform.rotate(img, angle, mode='reflect')

def flip(img, flip_h, flip_v):
    if flip_h:
        img = img[::-1]
    if flip_v:
        img = img[:, ::-1]
    return img

def rescale(img, scale_factor):
    zoomed_img = np.zeros_like(img, dtype=img.dtype)
    zoomed = skimage.transform.rescale(img, scale_factor)

    if scale_factor >= 1.0:
        shift_x = (zoomed.shape[0] - img.shape[0]) // 2
        shift_y = (zoomed.shape[1] - img.shape[1]) // 2
        zoomed_img[:,:] = zoomed[shift_x:shift_x+img.shape[0], shift_y:shift_y+img.shape[1]]
    else:
        shift_x = (img.shape[0] - zoomed.shape[0]) // 2
        shift_y = (img.shape[1] - zoomed.shape[1]) // 2
        zoomed_img[shift_x:shift_x+zoomed.shape[0], shift_y:shift_y+zoomed.shape[1]] = zoomed
        
    return zoomed_img

def crop(path, x1,y1, shape, crop_shape):
    x = plt.imread(path)
    x = x[x1:x1+crop_shape[0], y1:y1+crop_shape[1]]
    x = skimage.transform.resize(x, shape)
    return x
    
def prepare_images(dataframe, shape=IMG_SHAPE, crop_shape=CROP_SHAPE):
    x1 = (ORIG_SHAPE[0]-CROP_SHAPE[0])//2
    y1 = (ORIG_SHAPE[1]-CROP_SHAPE[1])//2
   
    sel = dataframe.values
    ids = sel[:,0].astype(int).astype(str)
    y_batch = sel[:,1:]
    x_batch = []
    for i in tqdm(ids):
        x = crop('../input/44352/images_training_rev1/'+i+'.jpg', x1,y1, shape=shape, crop_shape=crop_shape)
        
        degree = np.random.randint (0, 360)
        rotate(x, degree)
        
        zoom_scale = np.random.choice([1.3, 1.2, 1.1, 1, 1/1.1, 1/1.2, 1/1.3])
        rescale(x, zoom_scale)
        
        head, tail = np.random.choice([1, 0], 2)
        flip(x, head, tail)
        
        x /= 255.
        x_batch.append(x)
    x_batch = np.array(x_batch)
    return x_batch, y_batch
        
X_train, y_train = prepare_images(df_train)
#X_val, y_val = prepare_images(df_val)
X_test, y_test = prepare_images(df_test)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, GlobalMaxPooling2D
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

model = Sequential()
model.add(Conv2D(512, (3, 3), input_shape=(IMG_SHAPE[0], IMG_SHAPE[1], 3)))
model.add(Conv2D(512, (3, 3)))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(GlobalMaxPooling2D())


model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(37))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=[root_mean_squared_error])
model.summary()

In [ ]:
batch_size = 128
model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

# Test Prediction Submission